In [1]:
!pip install openai

# Database options
!pip install chromadb # if you use chromadb as your vector database

# Others
!pip install langchain-community # if you use langchain for orchastration
!pip install transformers #if you use huggingface for vector embedding

In [2]:
!pip install sentence-transformers

In [3]:
# enable GPU if needed, GPU can speed up your vector embedding if you computing these vectors locally (not using API)

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [4]:

import os
import json
import chromadb
import openai
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA


In [5]:
folder_path = "sample/"# Google drive path of the dataset
files_in_folder = os.listdir(folder_path)

len(files_in_folder)

1

In [6]:
import json

# file paths
json_path = "sample/amsfulfillment.com.json"

# JSON loading
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# extract desired text(contents inside 'text_by_page_url')
page_texts = data.get("text_by_page_url", {})


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import Document


In [9]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# download model in local and execute
pipe = pipeline("text-generation", model="google/flan-t5-base", max_new_tokens=128)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPT

In [11]:
from transformers import pipeline
text="\n".join(page_texts.values())
ner = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)
entities = ner(text[:1000])  # we usually limit for performance

for e in entities:
    print(e)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/opt/conda/lib/python3.12/site-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


{'entity_group': 'ORG', 'score': np.float32(0.79707897), 'word': 'Services', 'start': 34, 'end': 42}
{'entity_group': 'ORG', 'score': np.float32(0.7416121), 'word': 'Services', 'start': 148, 'end': 156}
{'entity_group': 'ORG', 'score': np.float32(0.89596707), 'word': 'Co', 'start': 173, 'end': 175}
{'entity_group': 'ORG', 'score': np.float32(0.8455391), 'word': '& P', 'start': 178, 'end': 181}
{'entity_group': 'ORG', 'score': np.float32(0.55324394), 'word': 'Shipping Services', 'start': 188, 'end': 205}
{'entity_group': 'ORG', 'score': np.float32(0.74804735), 'word': '##ting Services', 'start': 247, 'end': 260}
{'entity_group': 'ORG', 'score': np.float32(0.6292432), 'word': 'Systems & Technology', 'start': 300, 'end': 320}
{'entity_group': 'ORG', 'score': np.float32(0.7977522), 'word': 'Beauty Products & Cosmetics', 'start': 530, 'end': 557}
{'entity_group': 'ORG', 'score': np.float32(0.9551946), 'word': 'B Corp', 'start': 837, 'end': 843}
{'entity_group': 'ORG', 'score': np.float32(0.

In [12]:
import tempfile
from sklearn.cluster import KMeans
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
import re

persist_directory = tempfile.mkdtemp()

sentences = re.split(r'(?<=[.!?])\s+', text.strip())

embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
sentence_embeddings = embedding.embed_documents(sentences)

num_clusters = max(2, len(sentences) // 5)  # adjust cluster size as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(sentence_embeddings)

clustered_texts = {}
for label, sentence in zip(labels, sentences):
    clustered_texts.setdefault(label, []).append(sentence)

semantic_chunks = [" ".join(clustered_texts[label]) for label in sorted(clustered_texts.keys())]
documents = [Document(page_content=chunk) for chunk in semantic_chunks]

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory
)

pipe = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

retriever = vector_db.as_retriever(search_kwargs={"k": 2})
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

query = "What companies uses packaging materials in Valencia, California?"
# query="Where are the other branches of this company?"
response = qa.invoke(query)
print(response)

query1 ="Where is the headquarters of this company located?"
response1 = qa.invoke(query1)
print(response1)


/tmp/ipykernel_3269/3375455474.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
Device set to use cuda:0


{'query': 'What companies uses packaging materials in Valencia, California?', 'result': 'AMS Fulfillment Headquarters'}
{'query': 'Where is the headquarters of this company located?', 'result': 'Valencia, CA'}
